Desafio 1

copiar o arquivo para o container
docker cp arquivos/ namenode:/
conferir pelo docker
docker exec -it namenode ls /arquivos

In [105]:
#criar pasta HDFS de forma recursiva
!hdfs dfs -mkdir -p /user/marcos/data/covid

In [106]:
#copiar arquivos para o HDFS (rodar no namenode)
!hdfs dfs -put /arquivos/*.* /user/marcos/data/covid/

put: `/arquivos/*.*': No such file or directory


In [107]:
#listar pasta
!hdfs dfs -ls /user/marcos/data/covid

Found 5 items
-rw-r--r--   3 root supergroup   62499490 2022-04-23 13:07 /user/marcos/data/covid/HIST_PAINEL_COVIDBR_2020_Parte1_22abr2022.csv
-rw-r--r--   3 root supergroup   76521429 2022-04-23 13:07 /user/marcos/data/covid/HIST_PAINEL_COVIDBR_2020_Parte2_22abr2022.csv
-rw-r--r--   3 root supergroup   91134722 2022-04-23 13:07 /user/marcos/data/covid/HIST_PAINEL_COVIDBR_2021_Parte1_22abr2022.csv
-rw-r--r--   3 root supergroup   93415943 2022-04-23 13:07 /user/marcos/data/covid/HIST_PAINEL_COVIDBR_2021_Parte2_22abr2022.csv
-rw-r--r--   3 root supergroup   56680450 2022-04-23 13:07 /user/marcos/data/covid/HIST_PAINEL_COVIDBR_2022_Parte1_22abr2022.csv


In [108]:
!hdfs dfs -ls  /user/hive/warehouse/covid.db/covid

Found 5 items
-rwxrwxr-x   3 root supergroup   62499490 2022-04-23 12:59 /user/hive/warehouse/covid.db/covid/HIST_PAINEL_COVIDBR_2020_Parte1_22abr2022.csv
-rwxrwxr-x   3 root supergroup   76521429 2022-04-23 12:59 /user/hive/warehouse/covid.db/covid/HIST_PAINEL_COVIDBR_2020_Parte2_22abr2022.csv
-rwxrwxr-x   3 root supergroup   91134722 2022-04-23 12:59 /user/hive/warehouse/covid.db/covid/HIST_PAINEL_COVIDBR_2021_Parte1_22abr2022.csv
-rwxrwxr-x   3 root supergroup   93415943 2022-04-23 12:59 /user/hive/warehouse/covid.db/covid/HIST_PAINEL_COVIDBR_2021_Parte2_22abr2022.csv
-rwxrwxr-x   3 root supergroup   56680450 2022-04-23 12:59 /user/hive/warehouse/covid.db/covid/HIST_PAINEL_COVIDBR_2022_Parte1_22abr2022.csv


In [2]:
from pyspark.sql.functions import *
from pyspark.sql.types import IntegerType

In [3]:
#Leitura de arquivos
dados = spark.read.csv("/user/marcos/data/covid",sep=";",header="true")

In [4]:
dados.count()
#dados.select(dados.columns[1:3]).show()

4254754

In [5]:
#cast de dados
cast_int = dados.withColumn("recuperadosnovos",dados["recuperadosnovos"].cast('int'))\
.withColumn("emacompanhamentonovos",dados["emacompanhamentonovos"].cast('int'))\
.withColumn("casosacumulado",dados["casosacumulado"].cast('bigint'))\
.withColumn("obitosnovos",dados["obitosnovos"].cast('bigint'))\
.withColumn("casosnovos",dados["casosnovos"].cast('int'))\
.withColumn("populacaotcu2019",dados["populacaotcu2019"].cast('int'))



In [6]:
#Filtro apenas ultima data
ultimo_dia =  cast_int.filter('data = "2022-04-22"').filter('regiao = "Brasil"')

In [235]:
#jeito mais trabalhoso pois nao tem função max nativa
#visualizacao.agg(sum("recuperadosnovos").alias('Casos recuperados'),sum("emacompanhamentonovos").alias("Em acompanhamento")).show()

In [7]:
#Resp 1
visualizacao1 = ultimo_dia.groupBy().agg(max("recuperadosnovos").alias('casos_recuperados'),max("emacompanhamentonovos").alias("em_acompanhamento"))

visualizacao1.show()
#("casosacumulado",col('casosacumulado') * 1000000 / col('populacaotcu2019'))..cast('int')

+-----------------+-----------------+
|casos_recuperados|em_acompanhamento|
+-----------------+-----------------+
|         29352832|           323308|
+-----------------+-----------------+



In [ ]:
visualizacao1.write.saveAsTable("visualizacao1")

In [8]:
visualizacao2 = ultimo_dia.withColumn("incidencia",col('casosacumulado')*1000000 / col('populacaotcu2019')).select('casosacumulado','casosnovos','incidencia')
visualizacao2.show()

+--------------+----------+-----------------+
|casosacumulado|casosnovos|       incidencia|
+--------------+----------+-----------------+
|      30338697|      8068|144368.8415913375|
+--------------+----------+-----------------+



In [ ]:
visualizacao2.write.parquet("visualizacao2.parquet")

In [9]:
visualizacao3 = ultimo_dia.withColumn("letalidade",col('obitosacumulado')/100 * col('casosacumulado'))\
.withColumn("mortalidade",(col('obitosacumulado')* 1000000) / col('populacaotcu2019'))\
.select('obitosacumulado','obitosnovos','letalidade','mortalidade')
visualizacao3.show()

+---------------+-----------+--------------------+-----------------+
|obitosacumulado|obitosnovos|          letalidade|      mortalidade|
+---------------+-----------+--------------------+-----------------+
|         662557|         51|2.010111606822899...|3152.824479516434|
+---------------+-----------+--------------------+-----------------+



In [44]:
type(visualizacao3)

pyspark.sql.dataframe.DataFrame

In [68]:
from pyspark.sql.functions import *

In [75]:
#estruturar
visualizacao3_kafka = visualizacao3.select(to_json(struct('obitosacumulado','obitosnovos','letalidade','mortalidade')).alias("value"))


In [77]:
#Salvar no tópico kafka
visualizacao3_kafka_output = visualizacao3_kafka.write\
.format("kafka")\
.option("kafka.bootstrap.servers","kafka:9092")\
.option("topic","topic-kvspark")\
.save()

In [78]:
ultimo_dia.groupBy("regiao").sum("casosacumulado").show()

+------+-------------------+
|regiao|sum(casosacumulado)|
+------+-------------------+
|Brasil|           30338697|
+------+-------------------+



In [107]:
#Filtro apenas ultima data regiões
ultimo_dia_regioes =  cast_int.filter('data = "2022-04-22"')
#.filter('estado = "DF"')
#ultimo_dia_regioes.show()

In [108]:
grafico7 = ultimo_dia_regioes.withColumn("incidencia",col('casosacumulado')*1000000 / col('populacaotcu2019'))\
.withColumn("mortalidade",(col('obitosacumulado')* 1000000) / col('populacaotcu2019'))\
.select(col('casosacumulado').alias('casos'),col('obitosacumulado').alias('obtos'),col('incidencia').alias('Incidência_100mil_hab'),'mortalidade',col('data').alias("atualizacao")).show()


+--------+------+---------------------+------------------+-----------+
|   casos| obtos|Incidência_100mil_hab|       mortalidade|atualizacao|
+--------+------+---------------------+------------------+-----------+
|30338697|662557|    144368.8415913375| 3152.824479516434| 2022-04-22|
|  400317|  7197|   225248.35065903305|4049.5716636891784| 2022-04-22|
|  124633|  1998|    141317.6708034039|2265.4730790817916| 2022-04-22|
|  581934| 14169|    140407.8611261843|3418.6677257161555| 2022-04-22|
|  155391|  2147|   256521.96163173264|3544.3021257558676| 2022-04-22|
|  762630| 18215|    88648.37469842895|2117.3178935157066| 2022-04-22|
|  160375|  2130|    189628.8536189403|2518.5313060535796| 2022-04-22|
|  304206|  4149|   193408.72013254784|2637.8598049674924| 2022-04-22|
|  430626| 10879|    60864.30863040818|1537.6285073130991| 2022-04-22|
|  368022|  7735|   112433.99861971076|2363.1113882416344| 2022-04-22|
| 1243770| 26867|   136197.91683776682| 2942.046706127565| 2022-04-22|
|  502